In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.ticker import MaxNLocator
%matplotlib inline
plt.ion();
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
import utils_healpy as uhp
import utils_optimizers as uopt

def fitness_function(dataset, opt_params):
    target_rms = opt_params["fitness_desired_rms"]
    norm_factor = opt_params["fitness_norm_factor"]
    number_of_timesteps = np.shape(dataset["rms"][:,:])[1]

    rms = np.sqrt(np.sum(dataset["rms"][:,:]**2, axis=1) / float(number_of_timesteps))
    avg_flux = np.sqrt(np.sum(dataset["avg_flux"][:,:]**2, axis=1) / float(number_of_timesteps))
    if opt_params["run_plasma_profile"]:
        target_flux = opt_params["fitness_desired_pressure_mbar"]
        indices = np.where(np.array(avg_flux) > opt_params["fitness_limit_broken_pressure_mbar"])[0]
        #print("Fitness function detects broken runs: ", indices)
        avg_flux[indices] = 0.0
    else:
        target_flux = opt_params["fitness_desired_power_per_steradian"]

    maxi_func = np.exp(-(rms/target_rms) + (avg_flux / target_flux) ** 0.33) * (avg_flux / target_flux)**0.01 * norm_factor
    #maxi_func = np.exp(-(rms/target_rms) + (avg_flux / target_flux) ** 0.25) * (avg_flux / target_flux)**0.01 * norm_factor
    return maxi_func

def fitness_function_time_dependant(dataset, opt_params):
    target_rms = opt_params["fitness_desired_rms"]
    norm_factor = opt_params["fitness_norm_factor"]
    number_of_timesteps = np.shape(dataset["rms"][:,:])[1]

    rms = dataset["rms"][:,:]
    avg_flux = dataset["avg_flux"][:,:]
    if opt_params["run_plasma_profile"]:
        target_flux = opt_params["fitness_desired_pressure_mbar"]
        indices1 = np.where(np.array(avg_flux) > opt_params["fitness_limit_broken_pressure_mbar"])[0]
        avg_flux[indices1] = 0.0
        indices = np.where(np.array(avg_flux) < opt_params["fitness_limit_broken_pressure_mbar"] * 0.001)[0]
        print("Fitness function detects broken runs: ", indices1, indices)
    else:
        target_flux = opt_params["fitness_desired_power_per_steradian"]

    maxi_func = np.exp(-(rms/target_rms) + (avg_flux / target_flux) ** 0.25) * (avg_flux / target_flux)**0.01 * norm_factor
    return maxi_func

In [ ]:
"""
list_labels = [r"$\Omega$60 no CBET",
               r"$\Omega$60 CBET",
               r"$\Omega$60 varying $\Delta \lambda$",]
list_dirs = ["/ccc/scratch/cont002/dam/barlodun/2504a_PDDOptimization/Data/250429c_lafon351_omega_plasma_more",
             "/ccc/scratch/cont002/dam/barlodun/2504a_PDDOptimization/Data/250429b_lafon351_omega_plasma_cbet",
             "/ccc/scratch/cont002/dam/barlodun/2504a_PDDOptimization/Data/250507a_lafon351_omega_plasma_bandwidth_14ns",]
#"""
"""
list_labels = ["CPM48 no CBET",
               "CPM48 CBET",
               r"CPM48 $\Delta \lambda$ = 1%",]
list_dirs = ["/ccc/scratch/cont002/dam/barlodun/2504a_PDDOptimization/Data/250506i_cpm48_plasma",
             "/ccc/scratch/cont002/dam/barlodun/2504a_PDDOptimization/Data/250521f_cpm48_plasma_cbet",
             "/ccc/scratch/cont002/dam/barlodun/2504a_PDDOptimization/Data/250513g_cpm48_plasma_vary_beamspot_with_bandwidth_partial_fail",]
#"""
#"""
list_labels = ["CPM48",
               "CPM48 572nm",
               r"$\Omega$60 optimised",
               "ICO80",
               "T11_B72",
               "CPM72 optimised",]
list_dirs = [
             "/ccc/scratch/cont002/dam/barlodun/2504a_PDDOptimization/Data/250506i_cpm48_plasma",
             "/ccc/scratch/cont002/dam/barlodun/2504a_PDDOptimization/Data/250521d_527nm_5MJ_cpm48_plasma",
             "/ccc/scratch/cont002/dam/barlodun/2504a_PDDOptimization/Data/250429c_lafon351_omega_plasma_more_optimize",
             "/ccc/scratch/cont002/dam/barlodun/2504a_PDDOptimization/Data/250506a_ico80_plasma",
             "/ccc/scratch/cont002/dam/barlodun/2504a_PDDOptimization/Data/250506d_t11_b72_plasma",
             "/ccc/scratch/cont002/dam/barlodun/2504a_PDDOptimization/Data/250805a_cpm72_plasma_optimize",]
#"""
plot_file_type = ".png"

aspect_ratio = (5, 4)


In [ ]:
ndirs = len(list_labels)
sys_params = {}
dataset = {}
dataset_params = {}
deck_gen_params = {}
facility_spec = {}
opt_params = {}
fitness_temporal = {}
fitness_overall = {}
label_beam_config = {}

for idir in range(ndirs):
    label = list_labels[idir]
    loc_dir = list_dirs[idir]
    sys_params[label] = {}
    dataset[label] = {}
    dataset_params[label] = {}
    deck_gen_params[label] = {}
    facility_spec[label] = {}
    opt_params[label] = {}
    fitness_temporal[label] = {}
    fitness_overall[label] = {}

    print(label, loc_dir)
    sys_params[label] = tdg.define_system_params(loc_dir)
    dataset[label], dataset_params[label], deck_gen_params[label], facility_spec[label] = idg.load_data_dicts_from_file(sys_params[label])
    opt_params[label] = uopt.define_optimizer_parameters(loc_dir, 0, 0, dataset_params[label], facility_spec[label], sys_params[label])
    print(dataset[label]["rms"][0])

    fitness_temporal[label] = fitness_function_time_dependant(dataset[label], opt_params[label])
    fitness_overall[label] = fitness_function(dataset[label], opt_params[label])

    label_beam_config[label] = [None] * dataset_params[label]["num_examples"]
    for iconfig in range(dataset_params[label]["num_examples"]):
        label_beam_config[label][iconfig] = r"SG={:.2f} and $R_b/R_t$={:.2f} ".format(deck_gen_params[label]["beamspot_order"][iconfig,0], deck_gen_params[label]["beamspot_major_radius"][iconfig,0]/dataset_params[label]['target_radius'])
    label_beam_config[label] = np.array(label_beam_config[label])

In [ ]:
list_colours = [
                "tab:purple",
                "tab:pink",
                "tab:green",
                "tab:orange",
                "tab:blue",
                "tab:cyan",
                "tab:olive",
                "tab:gray",
                "tab:brown",
               ]
list_line_styles = ["solid","dotted","dashed","dashdot","solid","dotted","dashed","dashdot"]
list_point_styles = matplotlib.lines.Line2D.markers
print(list_point_styles)

In [ ]:
fig1, ax1 = plt.subplots(figsize=aspect_ratio, dpi=400)

ind_max_fitness = {}
ind_max_fitness_list = {}
ind_max_fitness_temp = {}
ind_min_rms_early_time = {}
rms_overall = {}

for idir in range(ndirs):
    label = list_labels[idir]
    number_of_timesteps = np.shape(dataset[label]["rms"])[1]
    
    ind_max_fitness[label] = np.argmax(fitness_overall[label])
    ind_max_fitness_list[label] = np.array(np.where(fitness_overall[label]>0.5)[0], dtype='int')
    print(label, len(ind_max_fitness_list[label]), np.max(fitness_overall[label]), list_point_styles[idir+3])
    rms_overall[label] = np.sqrt(np.sum(dataset[label]["rms"][:,:]**2, axis=1) / float(number_of_timesteps))
    
    facility_best_bs_rms_overall = rms_overall[label][ind_max_fitness_list[label]]
    facility_best_bs_ablation_pressure_late_time = dataset[label]["avg_flux"][ind_max_fitness_list[label],-1]
    
    ind_max_fitness_temp[label] = np.array(np.argmax(fitness_temporal[label][:,-1]), dtype='int')
    facility_best_late_time_bs_rms = dataset[label]["rms"][ind_max_fitness_temp[label],-1]
    facility_best_late_time_bs_rms_early_time = dataset[label]["rms"][ind_max_fitness_temp[label],0]
    facility_best_late_time_bs_ablation_pressure = dataset[label]["avg_flux"][ind_max_fitness_temp[label],-1]
    
    facility_best_early_time_bs_rms = np.min(dataset[label]["rms"][:,0][np.where(dataset[label]["avg_flux"][:,0]>0.0001)[0]])
    ind_min_rms_early_time[label] = np.where(dataset[label]["rms"][:,0]==facility_best_early_time_bs_rms)[0]
    facility_best_early_time_bs_pabl = dataset[label]["avg_flux"][ind_min_rms_early_time[label],0]

    ax1.semilogx(facility_best_bs_rms_overall*100, facility_best_bs_ablation_pressure_late_time, "o",
                markersize=5, color=list_colours[idir])#, label=label)
    ax1.semilogx(facility_best_late_time_bs_rms*100, facility_best_late_time_bs_ablation_pressure, "x",
                markersize=5, color=list_colours[idir])
    ax1.semilogx(facility_best_early_time_bs_rms*100, facility_best_early_time_bs_pabl, "^",
                markersize=5, color=list_colours[idir])
    
    sort_key = np.argsort(facility_best_bs_rms_overall)
    line_valx = [facility_best_bs_rms_overall[sort_key[0]]]
    line_valy = [facility_best_bs_ablation_pressure_late_time[sort_key[0]]]
    for ind in range(len(sort_key)):
        if facility_best_bs_ablation_pressure_late_time[sort_key[ind]]>line_valy[-1]:
            line_valx.append(facility_best_bs_rms_overall[sort_key[ind]])
            line_valy.append(facility_best_bs_ablation_pressure_late_time[sort_key[ind]])
    line_valx = np.array(line_valx)
    line_valy = np.array(line_valy)
    ax1.semilogx(line_valx * 100.0, line_valy, color=list_colours[idir], label=label)
    """ 
    ax1.text(rms_overall[label][ind_max_fitness[label]]*100, dataset[label]["avg_flux"][ind_max_fitness[label],-1],
                     label_beam_config[label][ind_max_fitness[label]], fontsize=5, rotation=45)
    ax1.text(facility_best_late_time_bs_rms*100, facility_best_late_time_bs_ablation_pressure,
                     label_beam_config[label][ind_max_fitness_temp[label]], fontsize=5, rotation=45)
    ax1.text(dataset[label]["rms"][ind_min_rms_early_time[label],0]*100, dataset[label]["avg_flux"][ind_min_rms_early_time[label],0],
                     label_beam_config[label][ind_min_rms_early_time[label]][0], fontsize=5, rotation=45)
    
    #"""
    """
    for ind in range(len(ind_max_fitness[label])):
        if (ind % 4 == 0):
            ax1.text(facility_best_bs_rms_overall[label][ind]*100, facility_best_bs_ablation_pressure_late_time[label][ind],
                     label_beam_config[label][ind_max_fitness[label][ind]], fontsize=5)
    """

ax1.legend(loc="lower right")

ax1.set_xlim([0.2,10])
ax1.set_ylim([0,220])
ax1.set_xlabel("RMS ablation pressure (% mean)")
ax1.set_ylabel("Ablation pressure (Mbars)")
fig1.savefig(loc_dir+"/../../"+sys_params[label]["figure_location"]+"/facility_comparison_pabl_v_rms" + plot_file_type, dpi=300, bbox_inches='tight')


In [ ]:
fig1, ax1 = plt.subplots(figsize=aspect_ratio, dpi=400)

ind_max_fitness = {}
ind_max_fitness_list = {}
ind_max_fitness_temp = {}
ind_min_rms_early_time = {}


for idir in range(2):
    label = list_labels[idir]
    number_of_timesteps = np.shape(dataset[label]["rms"])[1]
    ind_max_fitness[label] = np.argmax(fitness_overall[label])
    ind_max_fitness_list[label] = np.array(np.where(fitness_overall[label]>0.5)[0], dtype='int')
    print(label, len(ind_max_fitness_list[label]), np.max(fitness_overall[label]), list_point_styles[idir+3])
    rms_overall[label] = np.sqrt(np.sum(dataset[label]["rms"][:,:]**2, axis=1) / float(number_of_timesteps))
    
    facility_best_bs_rms_overall = rms_overall[label][ind_max_fitness_list[label]]
    facility_best_bs_ablation_pressure_late_time = dataset[label]["avg_flux"][ind_max_fitness_list[label],-1]

    ind_max_fitness_temp[label] = np.array(np.argmax(fitness_temporal[label][:,-1]), dtype='int')
    facility_best_late_time_bs_rms = dataset[label]["rms"][ind_max_fitness_temp[label],-1]
    facility_best_late_time_bs_rms_early_time = dataset[label]["rms"][ind_max_fitness_temp[label],0]
    facility_best_late_time_bs_ablation_pressure = dataset[label]["avg_flux"][ind_max_fitness_temp[label],-1]
    
    facility_best_early_time_bs_rms = np.min(dataset[label]["rms"][:,0][np.where(dataset[label]["avg_flux"][:,0]>0.0001)[0]])
    ind_min_rms_early_time[label] = np.where(dataset[label]["rms"][:,0]==facility_best_early_time_bs_rms)[0]
    facility_best_early_time_bs_pabl = dataset[label]["avg_flux"][ind_min_rms_early_time[label],0]

    #ax1.semilogx(facility_best_bs_rms_overall*100, facility_best_bs_ablation_pressure_late_time, "o",
    #             markersize=5, label=label, color=list_colours[idir])
    
    sort_key = np.argsort(facility_best_bs_rms_overall)
    line_valx = [facility_best_bs_rms_overall[sort_key[0]]]
    line_valy = [facility_best_bs_ablation_pressure_late_time[sort_key[0]]]
    #print(line_valy)
    for ind in range(len(sort_key)):
        if facility_best_bs_ablation_pressure_late_time[sort_key[ind]]>line_valy[-1]:
            line_valx.append(facility_best_bs_rms_overall[sort_key[ind]])
            line_valy.append(facility_best_bs_ablation_pressure_late_time[sort_key[ind]])
    line_valx = np.array(line_valx)
    line_valy = np.array(line_valy)
    ax1.semilogx(line_valx * 100.0, line_valy, color=list_colours[idir], label=label)

idir = 2
label = list_labels[idir]
indices_most_spectral_lines = np.where(deck_gen_params[label]["bandwidth_num_spectral_lines"]==20)[0]
rms_overall_most_spectral_lines = rms_overall[label][indices_most_spectral_lines]
ablation_pressure_late_time_most_spectral_lines = dataset[label]["avg_flux"][indices_most_spectral_lines,-1]

ax1.semilogx(rms_overall_most_spectral_lines*100, ablation_pressure_late_time_most_spectral_lines, "o",
             markersize=5, label=label, color=list_colours[idir])

num_ind = len(indices_most_spectral_lines)
label_bandwidth_beam_config = [None] * num_ind
for ind in range(num_ind):
    label_bandwidth_beam_config[ind] = r"$\Delta \lambda $={:.1f}%".format(deck_gen_params[label]["bandwidth_percentage_width"][indices_most_spectral_lines[ind],0])
    print(rms_overall_most_spectral_lines[ind]*100, ablation_pressure_late_time_most_spectral_lines[ind],
             label_bandwidth_beam_config[ind])
    horizontalalignment = "left"
    if ind == (num_ind-1):
        horizontalalignment = "right"
    ax1.text(rms_overall_most_spectral_lines[ind]*100, ablation_pressure_late_time_most_spectral_lines[ind],
             label_bandwidth_beam_config[ind], fontsize=10, horizontalalignment=horizontalalignment)
    
ax1.legend(loc="lower right")

ax1.set_xlim([0.2,10])
ax1.set_ylim([0,220])
ax1.set_xlabel("RMS ablation pressure (% mean)")
ax1.set_ylabel("Ablation pressure (Mbars)")
fig1.savefig(loc_dir+"/../../"+sys_params[label]["figure_location"]+"/facility_comparison_pabl_v_rms_bandwidth_omega" + plot_file_type, dpi=300, bbox_inches='tight')


In [ ]:
fig2, ax2 = plt.subplots(figsize=aspect_ratio, dpi=400)

ind_max_fitness = {}
ind_max_fitness_list = {}
ind_max_fitness_temp = {}
ind_min_rms_early_time = {}
rms_overall = {}

for idir in range(ndirs):
    label = list_labels[idir]
    number_of_timesteps = np.shape(dataset[label]["rms"])[1]
    ind_max_fitness[label] = np.argmax(fitness_overall[label])
    ind_max_fitness_list[label] = np.array(np.where(fitness_overall[label]>0.5)[0], dtype='int')
    print(label, len(ind_max_fitness_list[label]), np.max(fitness_overall[label]), list_point_styles[idir+3])
    rms_overall[label] = np.sqrt(np.sum(dataset[label]["rms"][:,:]**2, axis=1) / float(number_of_timesteps))
    facility_best_bs_ablation_pressure_late_time = dataset[label]["avg_flux"][ind_max_fitness_list[label],-1]
    
    for ind in range(len(ind_max_fitness_list[label])):
        SG_order = int(deck_gen_params[label]["beamspot_order"][ind_max_fitness_list[label][ind],0])
        pert = 0.1
        ax2.scatter(deck_gen_params[label]["beamspot_order"][ind_max_fitness_list[label][ind],0]+pert*ind, 
                    deck_gen_params[label]["beamspot_major_radius"][ind_max_fitness_list[label][ind],0]/dataset_params[label]['target_radius'], 
                    s=20, marker=idir+3, c=fitness_overall[label][ind_max_fitness_list[label][ind]], clim = (1.0, 1.4))#, marker=list_point_styles[idir+4])#, marker=list_point_styles[SG_order])
        #fig1.colorbar(cmap, ax=ax1, label="Overall fitness")